# TVM 中的调度原语

**原作者**: [Ziheng Jiang](https://github.com/ZihengJiang)

TVM 用于高效构建 kernel 的领域特定语言。

在本教程中，将您展示如何通过 TVM 提供的各种原语调度计算。

In [1]:
import tvm
from tvm import te
import numpy as np

通常有几种方法可以计算相同的结果，但是，不同的方法会导致不同的局部性（locality）和性能。因此 TVM 要求用户提供如何执行名为 **Schedule** （调度）的计算。

**Schedule** 是一组用于变换程序中计算循环的计算变换。

In [2]:
# 声明一些变量以备以后使用
n = te.var("n")
m = te.var("m")
p = te.var("p")

调度可以从 ops 列表中创建，默认情况下，调度以 row-major 顺序的串行方式计算张量。

In [3]:
# 声明矩阵元素级的乘法
A = te.placeholder((m, n), name="A")
B = te.placeholder((m, n), name="B")
C = te.compute((m, n), lambda i, j: A[i, j] * B[i, j], name="C")
s = te.create_schedule([C.op])

`lower` 将计算从定义转换为实际的可调用函数。使用 `simple_mode=True` 参数，它将返回可读的 C like 语句，在这里使用它来打印调度结果。

In [4]:
tvm.lower(s, [A, B, C], simple_mode=True).show()

每个调度由多个阶段（Stage）组成，每个阶段表示一个运算的调度。

下面提供各种方法来调度每个阶段。

## split

`split` 可以通过 `factor` 将指定的轴分裂（split）为两个轴。

In [6]:
m = te.var("m")
A = te.placeholder((m,), name="A")
B = te.compute((m,), lambda i: A[i] * 2, name="B")

s = te.create_schedule(B.op)
xo, xi = s[B].split(B.op.axis[0], factor=32)
tvm.lower(s, [A, B]).show()

你也可以通过 `nparts` 分裂轴，它与 `factor` 分割轴相对。

In [7]:
m = te.var("m")
A = te.placeholder((m,), name="A")
B = te.compute((m,), lambda i: A[i], name="B")

s = te.create_schedule(B.op)
bx, tx = s[B].split(B.op.axis[0], nparts=32)
tvm.lower(s, [A, B], simple_mode=True).show()

## tile

`tile` 帮助你在两个轴上逐块（tile by tile）执行计算。

In [8]:
A = te.placeholder((m, n), name="A")
B = te.compute((m, n), lambda i, j: A[i, j], name="B")

s = te.create_schedule(B.op)
xo, yo, xi, yi = s[B].tile(B.op.axis[0], B.op.axis[1], x_factor=10, y_factor=5)
tvm.lower(s, [A, B], simple_mode=True).show()

## fuse

`fuse` 可以融合一个计算的两个连续轴。


In [9]:
A = te.placeholder((m, n), name="A")
B = te.compute((m, n), lambda i, j: A[i, j], name="B")

s = te.create_schedule(B.op)
# tile to four axes first: (i.outer, j.outer, i.inner, j.inner)
xo, yo, xi, yi = s[B].tile(B.op.axis[0], B.op.axis[1], x_factor=10, y_factor=5)
# then fuse (i.inner, j.inner) into one axis: (i.inner.j.inner.fused)
fused = s[B].fuse(xi, yi)
tvm.lower(s, [A, B], simple_mode=True).show()

## reorder

`reorder` 可以按指定的顺序重新排列坐标轴。


In [10]:
A = te.placeholder((m, n), name="A")
B = te.compute((m, n), lambda i, j: A[i, j], name="B")

s = te.create_schedule(B.op)
# tile to four axes first: (i.outer, j.outer, i.inner, j.inner)
xo, yo, xi, yi = s[B].tile(B.op.axis[0], B.op.axis[1], x_factor=10, y_factor=5)
# then reorder the axes: (i.inner, j.outer, i.outer, j.inner)
s[B].reorder(xi, yo, xo, yi)
tvm.lower(s, [A, B], simple_mode=True).show()

## bind

`bind` 可以将指定的轴与线程轴绑定，通常用于 gpu 编程。

In [11]:
A = te.placeholder((n,), name="A")
B = te.compute(A.shape, lambda i: A[i] * 2, name="B")

s = te.create_schedule(B.op)
bx, tx = s[B].split(B.op.axis[0], factor=64)
s[B].bind(bx, te.thread_axis("blockIdx.x"))
s[B].bind(tx, te.thread_axis("threadIdx.x"))
tvm.lower(s, [A, B], simple_mode=True).show()

## compute_at

对于由多个算子组成的调度，默认情况下 TVM 将分别计算根节点上的张量。

In [12]:
A = te.placeholder((m,), name="A")
B = te.compute((m,), lambda i: A[i] + 1, name="B")
C = te.compute((m,), lambda i: B[i] * 2, name="C")

s = te.create_schedule(C.op)
tvm.lower(s, [A, B, C], simple_mode=True).show()

`compute_at` 可以将 `B` 的计算移到 `C` 的计算的第一个轴上。

In [13]:
A = te.placeholder((m,), name="A")
B = te.compute((m,), lambda i: A[i] + 1, name="B")
C = te.compute((m,), lambda i: B[i] * 2, name="C")

s = te.create_schedule(C.op)
s[B].compute_at(s[C], C.op.axis[0])
tvm.lower(s, [A, B, C], simple_mode=True).show()

## compute_inline

`compute_inline` 可以将一个阶段标记为内联，然后将计算体扩展并插入到需要张量的地址。

In [14]:
A = te.placeholder((m,), name="A")
B = te.compute((m,), lambda i: A[i] + 1, name="B")
C = te.compute((m,), lambda i: B[i] * 2, name="C")

s = te.create_schedule(C.op)
s[B].compute_inline()
tvm.lower(s, [A, B, C], simple_mode=True).show()

## compute_root

`compute_root` 可以将一个阶段的计算移到 root。

In [15]:
A = te.placeholder((m,), name="A")
B = te.compute((m,), lambda i: A[i] + 1, name="B")
C = te.compute((m,), lambda i: B[i] * 2, name="C")

s = te.create_schedule(C.op)
s[B].compute_at(s[C], C.op.axis[0])
s[B].compute_root()
tvm.lower(s, [A, B, C], simple_mode=True).show()

## 小结

本教程介绍了 tvm 中的调度原语，允许用户轻松灵活地调度计算。

为了得到性能良好的 kernel 实现，一般的工作流程往往是：

- 通过一系列的运算来描述你的计算。
- 试着用原语来调度计算。
- 编译并运行以查看性能差异。
- 根据运行时的结果调整你的调度。